In [1]:
import requests
import json
import os

In [6]:
url = "https://create.demo.sas.com"

def _connect_to_instance(refresh_token, verification):
	auth_url = f"{url}/SASLogon/oauth/token"
	# reading long-lived refresh token from txt file

	payload=f'grant_type=refresh_token&refresh_token={refresh_token}'
	headers = {
	'Accept': 'application/json',
	'Content-Type': 'application/x-www-form-urlencoded',
	'Authorization': 'Basic c2FzLmNsaTo=',
	}

	response = requests.request("POST", auth_url, headers=headers, data=payload, verify=verification)
	access_token = response.json()['access_token']
	return access_token

def _generate_access_token(auth_code, verification): 
	server = f"{url}/SASLogon/oauth/token"

	payload = f'grant_type=authorization_code&code={auth_code}'
	headers = {
		'Accept': 'application/json',
		'Content-Type': 'application/x-www-form-urlencoded',
		'Authorization': 'Basic c2FzLmNsaTo='
	}

	response = requests.request("POST", server, headers=headers, data=payload, verify=verification)

	# Parse the response text
	response_json = json.loads(response.text)

	# Get the refresh token
	refresh_token = response_json['refresh_token']
	return refresh_token

def get_connection(verify=False):
	if 'refresh_token.txt' not in [i.name for i in os.scandir()]:
		with open('refresh_token.txt', 'w') as file:
			file.write('test')
	if verify:
		os.environ['CAS_CLIENT_SSL_CA_LIST'] = 'workspaces/c2/workshop/Viya-connection/ssemonthly-rootCA-Intermidiates_4CLI.pem'
		verify = 'ssemonthly-rootCA-Intermidiates_4CLI.pem'
	try:
		with open('refresh_token.txt', 'r') as token:
			refresh_token = token.read()
		access_token = _connect_to_instance(refresh_token, verify)

	except:
		print('https://create.demo.sas.com/SASLogon/oauth/authorize?client_id=sas.cli&response_type=code')
		auth_code = input('Please provide your access token by going to https://create.demo.sas.com/SASLogon/oauth/authorize?client_id=sas.cli&response_type=code:')
		refresh_token = _generate_access_token(auth_code, verify)
		with open('refresh_token.txt', 'w') as file:
			file.write(refresh_token)
		access_token = _connect_to_instance(refresh_token, verify)
	print('Connected!')
	return access_token

In [7]:
access_token = get_connection(verify=True)

Connected!


In [4]:
from sasctl import Session, pzmm
from sasctl.services import model_repository as mr
st = Session(url, token=access_token)
st

In [5]:
project_name = "Workbench GSI"
repository_name = "DMRepository"

repository = mr.get_repository(repository_name)

project = mr.get_project(project_name)

if project == None:
    project = mr.create_project(project_name, repository)

OSError: Could not find a suitable TLS CA certificate bundle, invalid path: workspaces/c2/workshop/Viya-connection/ssemonthly-rootCA-Intermidiates_4CLI.pem

In [17]:
import sasviya 
model = sasviya.load_model('../out_data/forest_model.pkl')

model_params = {
    "name": "RF_Model",
    "projectId": project.id,
    "type": "ASTORE",
    'function':'Classification'

}

astore = mr.post(
    "/models",
    files={"files": ("model_export.astore", model.export())},
    data=model_params,
)

FileNotFoundError: [Errno 2] No such file or directory: '../out_data/forest_model.pkl'

In [ ]:

# Get the input variables
def get_project_variables(base_path):
    """
        Returns an array containing the input and output variables
        Requires the inputVar.json and outputVar.json to be available
        in the base_path location
    """
    # Add the role of input to the input variables
    input_var_JSON = pd.read_json(f"{base_path}/inputVar.json")
    project_input_variables = list(pd.DataFrame.to_dict(input_var_JSON.transpose()).values())
    for var in project_input_variables:
        var['role'] = 'input'

    # Add the role of output to the output variables
    output_var_JSON = pd.read_json(f"{base_path}/outputVar.json")
    project_output_variables = list(pd.DataFrame.to_dict(output_var_JSON.transpose()).values())
    for var in project_output_variables:
        var['role'] = 'output'

    # Join all variables into an array to register with SAS Model Manager
    project_variables = project_input_variables + project_output_variables
    return project_variables

def get_model_attributes(base_path):
    """
        Returns an object containing the model attributes
    """   
    with open(f"{base_path}/modelConfiguration.json", 'r') as modelConfiguration:
        model_attributes = json.load(modelConfiguration)
    
    return model_attributes

def update_model_tags(model_attributes, model_id):
    model_details = mr.get_model_details(model_id)
    headers = {
            "Content-Type": "application/vnd.sas.models.model+json",
            "Accept": "application/vnd.sas.models.model+json",
            "If-Match": model_details._headers["ETag"]
        }
    model_response = dict(model_details.items())
    model_response['tags'] = model_attributes['tags']
    res = s.put(f'/modelRepository/models/{model_details.id}', data=json.dumps(model_response), headers=headers)
    if res.status_code == 200:
        print("The model tags have been updated")
    
def register_model(base_path):
    model_attributes = get_model_attributes(base_path)
    model_object = mr.create_model(model = model_attributes,
        project = project_attributes['project_name'])
    
    time.sleep(1)
    # Score script
    file = open(f"{base_path}/{model_attributes['scoreCodeFile']}", 'rb')
    mr.add_model_content(model_object,
                         file, 
                         name = model_attributes['name'] + '.py',
                         role = 'score')
    file.close()

    # Dependencies
    file = open(f"{base_path}/requirements.json", 'rb')
    mr.add_model_content(model_object,
                         file, 
                         name = 'requirements.json',
                         role = 'python pickle')
    file.close()

    # Output variables
    file = open(f"{base_path}/outputVar.json", 'rb')
    mr.add_model_content(model_object,
                         file,
                         name = 'outputVar.json')
    file.close()
    
    # Input variables
    file = open(f"{base_path}/inputVar.json", 'rb')
    mr.add_model_content(model_object,
                         file,
                         name = 'inputVar.json')
    file.close()

    # Options information
    file = open(f"{base_path}/options.json", 'rb')
    mr.add_model_content(model_object,
                         file,
                         name = 'options.json',
                         role='documentation')
    file.close()

    # Upload the optional model card
    if os.path.exists(f"{base_path}/Model-Card.pdf"):
        file = open(f"{base_path}/Model-Card.pdf", 'rb')
        mr.add_model_content(model_object,
                            file,
                            name = 'Model-Card.pdf',
                            role = 'documentation')
    elif os.path.exists(f"{base_path}/Model-Card.md"):
        file = open(f"{base_path}/Model-Card.md", 'rb')
        mr.add_model_content(model_object,
                            file,
                            name = 'Model-Card.md',
                            role = 'documentation')
    else:
        print(f"No model card added for {base_path}")

    update_model_tags(model_attributes, model_object.id)
    print(f"Link to the model in SAS Model Manager: {server}/SASModelManager/models/{model_object.id}")
    return model_object


model_name = register_model(model)

In [ ]:
{
    "name": "",
    "scoreCodeFile": "",
    "description": "",
    "toolVersion": "3.11-5",
    "targetVariable": "response",
    "targetLevel": "NOMINAL",
    "trainCodeType": "Python",
    "modeler": "",
    "function": "classification",
    "algorithm": "Transformer",
    "tool": "Python 3",
    "scoreCodeType": "Python",
    "champion": false,
    "tags": ["SLM", "Open-Source", "License", "Company", "size"]
}